In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import logging

In [3]:
data = pd.read_csv('../data/vacancies_ru.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
0,v_862116,Смотритель музейный,c_162972,"['Пользователь ПК', 'Работа в команде', 'Умени...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,NaN,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5,3
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,NaN,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5,1
4,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209567,v_2484959,Сборщик-упаковщик,c_203256,"['Пользователь ПК', 'Умение работать в команде...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience,Сборщик-упаковщик,74,4876,114733,5,2,1
2209568,v_205163,Сварщик на полуавтомат,c_158695,"['Желание работать и зарабатывать', 'Высокая э...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3,Сварщик на полуавтомат,71,5032,65219,5,5,2
2209569,v_639897,Главный инженер / Технический директор,c_209365,"['Контроль исправности оборудования', 'Инженер...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3,Главный инженер,41,842,121520,5,5,2
2209570,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,"['Предпечатная подготовка', 'Статистический ан...",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience,Фармацевт,62,2673,162497,5,5,1


In [81]:
# функции для маппинга superjob с hh.ru

In [4]:
data['workSchedule'].unique()

array(['fullDay', 'flyInFlyOut', 'flexible', 'shift', 'remote'],
      dtype=object)

In [5]:
data[data['workSchedule'] == 'flyInFlyOut'].head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
7,v_2091086,Монтажник СТиЖБК,c_230890,NaN,100000.0,NaN,RUR,a_500,ar_36,full,flyInFlyOut,between1And3,Монтажник СТиЖБК,35,4447,145437,5,1,2
23,v_2416809,Водитель автомобиля 6-го разряда (БЕЛАЗ -7555),c_220194,"['Умение работать в команде', 'Точность и вним...",138100.0,138100.0,RUR,a_3095,ar_20,full,flyInFlyOut,between3And6,Водитель автомобиля 6-го разряда,18,2330,133553,5,1,3
60,v_1070312,Водитель самосвала,c_18924,"['Водительское удостоверение категории BC', 'З...",145000.0,180000.0,RUR,a_4551,ar_28,full,flyInFlyOut,between1And3,Водитель самосвала,26,3948,99158,5,1,2
73,v_194247,Механик бурового оборудования,c_109998,"['Техническое обслуживание', 'Точность и внима...",250000.0,NaN,RUR,a_3445,ar_56,full,flyInFlyOut,between1And3,Механик бурового оборудования,57,2719,11111,5,1,2
77,v_1685474,Машинист бульдозера SHANTUI SD-16,c_13187,NaN,160000.0,NaN,RUR,a_500,ar_36,full,flyInFlyOut,between1And3,Машинист бульдозера SHANTUI SD-16,35,4447,35414,5,1,2


In [6]:
data[data['workSchedule'] == 'shift'].head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
27,v_484279,Кладовщик на склад (Артем),c_243786,"['Приемка груза', 'Документальное сопровождени...",70000.0,NaN,RUR,a_4931,ar_23,full,shift,between1And3,Кладовщик на склад,21,4370,159765,5,4,2
29,v_2226044,Специалист по вооруженному сопровождению грузов,c_49304,NaN,70000.0,NaN,RUR,a_1756,ar_41,full,shift,between1And3,Специалист по вооруженному сопровождению грузов,41,842,222586,5,4,2
30,v_2515950,Администратор отеля,c_170361,NaN,32000.0,35000.0,RUR,a_6223,ar_78,part,shift,noExperience,Администратор отеля,81,5806,78183,4,4,1
31,v_1254531,Мастер цеха по производству ЛКМ,c_91131,"['Пользователь ПК', 'Работа в команде', 'Контр...",70000.0,NaN,RUR,a_5527,ar_69,full,shift,between3And6,Мастер цеха по производству ЛКМ,71,5032,269061,5,4,3
36,v_1545695,Комплектовщик / сборщик заказов на склад обуви...,c_235224,NaN,47000.0,52000.0,RUR,a_5111,ar_64,full,shift,between1And3,Комплектовщик,66,4571,150253,5,4,2


In [7]:
data[data['workSchedule'] == 'remote'].head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
32,v_1284861,Менеджер по развитию В2В,c_160874,"['Грамотная речь', 'Деловая коммуникация', 'На...",42000.0,NaN,RUR,a_1756,ar_41,full,remote,between1And3,Менеджер по развитию В2В,41,842,67641,5,3,2
35,v_2494877,Ассистент отдела продаж,c_94905,"['Работа в команде', 'Федеральные сети', 'Знан...",30000.0,30000.0,RUR,a_1756,ar_41,full,remote,between1And3,Ассистент отдела продаж,41,842,273253,5,3,2
37,v_2230711,Преподаватель чешского языка,c_13772,"['Преподаватель', 'Грамотная речь']",40000.0,NaN,RUR,a_6223,ar_78,full,remote,between1And3,Преподаватель чешского языка,81,5806,41914,5,3,2
54,v_2491017,Специалист отдела телемаркетинга (Удаленно),c_233435,"['Активные продажи', 'Работа с базами данных',...",30000.0,50000.0,RUR,a_3405,ar_81,full,remote,between1And3,Специалист отдела телемаркетинга,85,2675,148265,5,3,2
58,v_1494003,Консультант телефонной поддержки в Яндекс Прак...,c_32656,NaN,30000.0,42000.0,RUR,a_2431,ar_64,full,remote,noExperience,Консультант телефонной поддержки в Яндекс Прак...,66,1593,204088,5,3,1


In [22]:
def get_employment(x):
    if x == 6: # полный день
        return 'full'
    elif x == 10 or x == 13: #неполный день или частичная занятость
        return 'part'
    elif x == 12 or x == 9: # сменный график или вахтовый метод
        return 'full'
    elif x == 7: # временная работа
        return 'project'
    else:
        return 'full'

In [23]:
def get_workSchedule(schedule_x, place_x):
    if place_x == 2:
        return 'remote'
    elif schedule_x == 6 or schedule_x == 7:
        return 'fullDay'
    elif schedule_x == 10 or schedule_x == 13:
        return 'flexible'
    elif schedule_x == 12:
        return 'shift'
    elif schedule_x == 9:
        return 'flyInFlyOut'
    else:
        return 'fullDay'

In [24]:
data['workExperience'].unique()

array(['noExperience', 'between3And6', 'between1And3', 'moreThan6'],
      dtype=object)

In [25]:
def get_workExperience(x):
    if x == 1:
        return 'noExperience'
    elif x == 2:
        return 'between1And3'
    elif x == 3:
        return 'between3And6'
    elif x == 4:
        return 'moreThan6'
    else:
        return 'noExperience'

In [181]:
# получения всех кодов каталогов

url_cats = 'https://api.superjob.ru/2.0/catalogues/'
response_cats = requests.get(url)

if response_cats.status_code == 200:
    res = response_cats.json()

In [182]:
res[0]

{'title_rus': 'IT, Интернет, связь, телеком',
 'url_rus': 'it-internet-svyaz-telekom',
 'title': 'IT, Интернет, связь, телеком',
 'title_trimmed': 'IT, Интернет, связь,...',
 'key': 33,
 'positions': [{'title_rus': 'CRM-системы',
   'url_rus': 'crm-sistemy',
   'title': 'CRM-системы',
   'id_parent': 33,
   'key': 603},
  {'title_rus': 'Data Science',
   'url_rus': 'data-science',
   'title': 'Data Science',
   'id_parent': 33,
   'key': 627},
  {'title_rus': 'DevOps',
   'url_rus': 'devops',
   'title': 'DevOps',
   'id_parent': 33,
   'key': 628},
  {'title_rus': 'SRE',
   'url_rus': 'sre',
   'title': 'SRE',
   'id_parent': 33,
   'key': 629},
  {'title_rus': 'Web-верстка',
   'url_rus': 'web-verstka',
   'title': 'Web-верстка',
   'id_parent': 33,
   'key': 36},
  {'title_rus': 'Администрирование баз данных',
   'url_rus': 'administrirovanie-baz-dannyh',
   'title': 'Администрирование баз данных',
   'id_parent': 33,
   'key': 37},
  {'title_rus': 'Аналитика',
   'url_rus': 'analit

In [183]:
len(res)

32

In [184]:
catalogues_codes = []
for i in range(len(res)):
    catalogues_codes.append(res[i]['key'])

In [186]:
# сбор датасета

In [187]:
columns = ['vacancy_id', 'name', 'company_id', 'keySkills', 'compensation_from',
           'compensation_to', 'compensation_currencyCode', 'area_id', 'employment',
           'workSchedule', 'workExperience', 'description', 'published_at']

In [190]:
url = 'https://api.superjob.ru/2.0/vacancies/'
API_KEY = 'v3.r.138336259.24bd22e30244a5ff349f72abfd00a5348a853998.b32035a7c7740e65e7ab2fcec46f96620e0005b7'

In [194]:
headers = {'X-Api-App-Id': API_KEY}

page_cnt = 2
period = 0
order = 'date'
count = 2

In [195]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

In [196]:
logging.basicConfig(
    filename='superjob_logs.log',
    level=logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(message)s'
    )

In [ ]:
df = pd.DataFrame(columns=columns)
ind = 0

for code in catalogues_codes:
    ind += 1
    
    progress_bar = tqdm(total=page_cnt, desc=f"Catalogue code {code}, {ind}/{len(catalogues_codes)}")
    
    for j in range(page_cnt):
        progress_bar.update(1)

        params = {
        #     'keyword': 'vac_name',
            'page': j, 'count': count, 'period': period, 'catalogues': code, 'order_field': order
        }

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:

            vacancies = response.json()['objects']

            for i in range(len(vacancies)):

                vac = vacancies[i]

                row = {column: None for column in columns}

                row['vacancy_id'] = vac.get('id')
                row['name'] = vac.get('profession')
                row['company_id'] = vac.get('id_client')
                row['keySkills'] = vac.get('candidat').replace('•', '') if vac.get('candidat') else None
                row['compensation_from'] = vac.get('payment_from')
                row['compensation_to'] = vac.get('payment_to')
                row['compensation_currencyCode']= 'RUR' if vac.get('currency') == 'rub' else None
                row['area_id'] = vac.get('town')['id'] if vac.get('town') else None # city
                row['employment'] = get_employment(vac['type_of_work'].get('id')) if vac.get('type_of_work') else None
                row['workSchedule'] = get_workSchedule(vac['type_of_work'].get('id'), vac['place_of_work'].get('id'))\
                                                                if vac['type_of_work'] and vac['place_of_work'] else None
                row['workExperience'] = get_workExperience(vac.get('experience').get('id')) if vac.get('experience') else None
                row['description'] = BeautifulSoup(vac.get('vacancyRichText'), 'html.parser').get_text(strip=True)
                row['published_at'] = datetime.fromtimestamp(vac.get('date_published')).strftime('%Y-%m-%d %H:%M:%S')
                row['alternate_url'] = vac.get('link')

                df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
                logging.info(f'Successful concatenation. Shape of dataframe: {df.shape}')

        else:
            logging.info(f'Error {response.status_code}')

    progress_bar.close()

Catalogue code 306, 26/32:   0%|                          | 0/2 [00:00<?, ?it/s]

In [177]:
with open('superjob_logs.log', 'r') as file:
    logs = file.readlines()

print(logs[0].strip())

2024-05-07 13:19:24,152 - INFO - Successful concatenation. Shape of dataframe: (1, 14)


In [180]:
df

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,employment,workSchedule,workExperience,description,published_at,alternate_url
0,48995056,Специалист технической поддержки,190447,"Обязанности:\n Регистрация, решение и эскалаци...",95000,0,RUR,1395,full,fullDay,between1And3,"Обязанности:Регистрация, решение и эскалация о...",2024-05-07 13:34:00,https://lyubertsy.superjob.ru/vakansii/special...
1,46559833,Инженер-технолог,173974,Обязанности:\n Соблюдение технических процессо...,46000,46000,RUR,1154,full,fullDay,between1And3,Обязанности:Соблюдение технических процессов в...,2024-05-07 13:35:00,https://verkhnij-ufalej.superjob.ru/vakansii/i...
2,49143494,Специалист по работе с клиентами,24172,Что нужно делать?\n Заключение договоров;\n Со...,51600,63600,RUR,14,full,shift,noExperience,Что нужно делать?Заключение договоров;Сохранен...,2024-05-07 13:33:49,https://spb.superjob.ru/vakansii/specialist-po...
3,49144192,Консультант контактного центра,4575811,"Телеконтакт, совместно с крупнейшим в мире меж...",32000,0,RUR,20,full,shift,noExperience,"Телеконтакт, совместно с крупнейшим в мире меж...",2024-05-07 13:39:09,https://lipetsk.superjob.ru/vakansii/konsultan...
4,49199546,Сервисный менеджер,4752098,Компания Твлинии занимается оптовыми продажами...,50000,0,RUR,12,full,fullDay,between1And3,Компания Твлинии занимается оптовыми продажами...,2024-05-07 13:41:37,https://nn.superjob.ru/vakansii/servisnyj-mene...
5,48658863,Менеджер по продажам продуктов торгового эквай...,297921,Сбер — лучший работодатель России. У нас 14 ты...,75000,0,RUR,141,full,fullDay,between1And3,Сбер — лучший работодатель России. У нас 14 ты...,2024-05-07 13:15:00,https://irkutsk.superjob.ru/vakansii/menedzher...
6,48393519,Старший менеджер по продажам,24172,Перед вами не просто вакансия. Перед вами — ун...,0,112000,RUR,389,full,fullDay,between1And3,Перед вами не просто вакансия. Перед вами — ун...,2024-05-07 13:30:08,https://stavropol.superjob.ru/vakansii/starshi...
7,49143376,Системный администратор / Инженер (IT-аутсорсинг),633788,Приглашаем к сотрудничеству специалистов IT (с...,0,0,RUR,920,full,shift,between1And3,Приглашаем к сотрудничеству специалистов IT (с...,2024-05-07 13:25:40,https://chusovoj.superjob.ru/vakansii/sistemny...
8,49299817,Инженер,4430,Обязанности: \n составление текущей отчетности...,65000,0,RUR,141,full,fullDay,between1And3,Обязанности:составление текущей отчетности по ...,2024-05-07 13:33:09,https://irkutsk.superjob.ru/vakansii/inzhener-...
9,43661535,Офис-менеджер / Секретарь / Администратор сайта,2387946,Обязанности:\n Прием и распределение входящих ...,60000,70000,RUR,4,full,fullDay,between3And6,Обязанности:Прием и распределение входящих зво...,2024-05-07 13:33:04,https://www.superjob.ru/vakansii/ofis-menedzhe...


In [179]:
df_new = df.dropna(subset=['name'])
df_new = df_new.drop_duplicates(subset=['vacancy_id'])

In [ ]:
df.shape

In [55]:
# draft to check fields

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    
    vac = response.json()['objects']
    
vac[0]['candidat']

'Для каждого, кто хочет работать и зарабатывать! Для каждого, кто готов к профессиональному и карьерному росту! Ведущая федеральная розничная сеть магазинов «Пятерочка» приглашает на работу:\n\n\nЧЕМ НУЖНО ЗАНИМАТЬСЯ:\n• Выпечка хлебобулочных изделий из полуфабрикатов\n• Упаковка, маркировка, выкладка\n• Работа за кассой и в торговом зале\n\n\nОТ ВАС:\n• Желание работать с хлебобулочными продуктами, остальному научим!\n\n\nОТ НАС:\n• Оформление по ТК РФ\n• Фиксированный оклад + премии и надбавки за стаж. Средний доход 45500 – 52000 руб. в месяц до вычета налогов\n• График работы 5/2, 2/2, возможен неполный рабочий день, неполная рабочая неделя\n• Финансовая поддержка в сложных жизненных ситуациях\n• Медицинская книжка за счет компании \n• Скидки в магазинах сети, скидки от партнеров\n• Оборудованная комната отдыха с бесплатным кофе и чаем\n• Медицинская страховка для сотрудника и его семьи в формате телемедицины, включая поддержку узких специалистов\n• Подарки и компенсация отдыха для 